## [미니프로젝트] 악성사이트 탐지 머신러닝 모델 개발

### 여러분은 기업 보안팀에서 근무중인 엔지니어로써, 웹페이지에서 추출한 Feature(특징) 기반으로 악성사이트를 탐지하는 머신러닝 모델 개발 미션을 부여받았습니다.

### ▣ 우리가 풀어야 하는 문제는 무엇인가요?
 - 웹 페이지에서 Feature를 추출하세요.
 - 악성사이트 여부를 판별하는 성능 좋은 AI모델을 생성하세요.

<br>

---

## ▣ 데이터 소개
* 웹 크롤링 데이터셋 : Feature_Website.xlsx

## ▣ 웹 크롤링 데이터셋의 변수 소개
* html_code : 크롤링을 활용해 수집한 HTML Code 원본
* repu : 악성사이트 여부 (malicious : 악성사이트, benign : 정상사이트)
<br>

---

## <b>[1단계] 데이터 수집</b>

* 1단계에서는 크롤링으로 수집한 HTML Code를 활용해 Feature를 만드는 과정을 체험합니다.

# <b>Step 0. 본격적인 실습 전 packages 설치
* Beautifulsoup 라이브러리 설치
* openpyxl 라이브러리 설치

In [169]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

* 데이터 프레임 관련 라이브러리 Import

In [170]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


---
## <b>Q1. 데이터 불러오기
### 정상/악성 HTML Code가 저장된 엑셀파일 불러오기
- 파일명 : Feature Website.xlsx


### <span style="color:pink">[문제1] Pandas 라이브러리를 활용해서 'Feature Website.xlsx'파일을 'df' 변수에 저장하고 그 info()및 head()를 통해 데이터를 확인하세요.<span>

In [171]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df = pd.read_excel('Feature_Website.xlsx')

In [172]:
# 데이터 프레임의 info를 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   html_code  40 non-null     object
 1   repu       40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [173]:
# 불러온 데이터를 확인합니다.
df

,html_code,repu
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious


---
# <b>Step 1. 데이터 수집

### 주어진 데이터로만 모델링 하는 경우는 드뭅니다.
### 주어진 데이터 외 추가로 데이터를 수집 또는 생성해야 하는 경우가 많습니다.
### 이번 과정에서는 웹 크롤러를 통해 수집된 정상/악성 사이트 HTML 데이터에서
### BeatifulSoup 라이브러리를 활용 필요한 Feature(특징)를 추출해 보도록 하겠습니다.
### 정상/악성 사이트 HTML Code는 사전에 수집하여 'Feature Website.xlsx' 파일에 저장해 두었습니다.


### <span style="color:skyblue">[예시] Beatuifulsoup 라이브러리를 활용 HTML code를 출력하고 \<title> 태그 길이를 계산합니다.<span>

In [174]:
soup = BeautifulSoup(df['html_code'][0], 'html.parser') # beautifulsoup을 이용해서 html.parser로 엑셀 파일의 저 데이터를 읽어서 soup에 넣음

*<span style="color:skyblue"> html code 출력<span>

In [175]:
print(soup) # parser로 읽어서 html코드를 우리가 다루기 편하게 만들어줌

<!DOCTYPE html>

<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->
<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->
<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER"

* <span style="color:skyblue"> \<title> 태그 출력 및 길이 계산<span>

In [176]:
# <title> 태그 출력
print("* title :",soup.head.title)

# <title> 태그 길이 출력
print("* title 길이 :", len(str(soup.head.title.getText())))

* title : <title dir="ltr">Amazon.com</title>
* title 길이 : 10


In [177]:
def title_length(soup):
    try:
        return (len(str(soup.head.title.getText()))) # 타이틀길이 반환 함수
    except:
        return 0.0 # 타이틀이 없으면 0 반환

In [178]:
title_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    title_len.append(float(title_length(soup)))

In [179]:
df['title_length'] = title_len

In [180]:
df

,html_code,repu,title_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,17.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,36.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,45.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,77.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,14.0


---

## <b>Q2. html 에서 \<script>...\</script> 태그 길이 계산
- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:pink">[문제2] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [219]:
print("* script 길이 :", len(str(soup.script)))

# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def script_length(soup):
    try:
        return (len(str(soup.script))) # 타이틀길이 반환 함수
    except:
        return 0.0 # 타이틀이 없으면 0 반환


* script 길이 : 268


In [220]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
script_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    script_len.append(float(script_length(soup)))

In [221]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['script_length'] = script_len
df


,html_code,repu,title_length,script_length,spacing_length,body_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0,518.0,402.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0,1419.0,1041.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0,729.0,433.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0,0.0,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0,3492.0,2969.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,17.0,3180.0,832.0,0.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,36.0,5056.0,1051.0,792.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,45.0,172.0,223.0,0.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,77.0,1389.0,2720.0,2032.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,14.0,119.0,76.0,0.0


---

## <b>Q3. html에서 공백 수 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:pink">[문제3] Beatuifulsoup 라이브러리를 활용 HTML Code에서 \<html> 태그 공백 수를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [184]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
# soup.body.text.count(' ')
print("* 태그 공백 수 :", len(str(soup.getText(' '))))
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def spacing_length(soup):
    try:
        return (len(str(soup.getText(' ')))) # 타이틀길이 반환 함수
    except:
        return 0.0 # 타이틀이 없으면 0 반환
    

* 태그 공백 수 : 154


In [185]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
spacing_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    spacing_len.append(float(spacing_length(soup)))
print(spacing_len)


[518.0, 1419.0, 729.0, 0.0, 3492.0, 832.0, 1051.0, 223.0, 2720.0, 76.0, 3447.0, 1159.0, 90.0, 6363.0, 357.0, 60.0, 164.0, 10353.0, 11037.0, 3367.0, 0.0, 142.0, 1246.0, 225.0, 96.0, 1395.0, 4863.0, 1269.0, 314.0, 855.0, 182.0, 76.0, 1387.0, 173.0, 179.0, 420.0, 1.0, 161.0, 9951.0, 154.0]


In [186]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['spacing_length'] = spacing_len
df

,html_code,repu,title_length,script_length,spacing_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,388.0,518.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,562.0,1419.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,800.0,729.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,0.0,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,408.0,3492.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,17.0,3163.0,832.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,36.0,4992.0,1051.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,45.0,128.0,223.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,77.0,1349.0,2720.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,14.0,102.0,76.0


---

## <b>Q4. html 에서 body 길이 계산

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:pink">[문제4] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<body> 태그 길이를 계산하는 함수를 완성하고 결과를 확인하세요.<span>

In [228]:

# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def body_length(soup):
    try:
        return (len(str(soup.body.getText()))) # 타이틀길이 반환 함수
    except:
        return 0.0 # 타이틀이 없으면 0 반환
    

In [229]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다.
body_len = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    body_len.append(float(body_length(soup)))
print(body_len)

[402.0, 1041.0, 433.0, 0.0, 2969.0, 0.0, 792.0, 0.0, 2032.0, 0.0, 2733.0, 888.0, 0.0, 5476.0, 273.0, 0.0, 139.0, 9297.0, 9951.0, 3047.0, 0.0, 0.0, 854.0, 0.0, 0.0, 0.0, 4168.0, 1078.0, 0.0, 322.0, 4.0, 0.0, 1005.0, 0.0, 0.0, 24.0, 0.0, 0.0, 8932.0, 0.0]


In [230]:
# 추출한 Feature(특징) 데이터를 확인합니다.
df['body_length'] = body_len
df


,html_code,repu,title_length,script_length,spacing_length,body_length
0,<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang=...,malicious,10.0,405.0,518.0,402.0
1,\n\t\n\n\n\t\n\n\t\n\n\n\t\n\n\n\t\n\n\t\n\t\t...,malicious,5.0,579.0,1419.0,1041.0
2,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\...",malicious,71.0,817.0,729.0,433.0
3,"<!DOCTYPE html><html lang=""en""><head><style da...",malicious,0.0,4.0,0.0,0.0
4,<!DOCTYPE html>\n\n\n \n \n \n \n ...,malicious,27.0,425.0,3492.0,2969.0
5,_x000D_\n_x000D_\n_x000D_\n<!DOCTYPE html>_x00...,malicious,17.0,3180.0,832.0,0.0
6,"<!doctype html>\n\n<html data-ytrk-page=""HOME""...",malicious,36.0,5056.0,1051.0,792.0
7,"\n\t<!DOCTYPE html>\n\t<html class=""no-icon-fo...",malicious,45.0,172.0,223.0,0.0
8,"<!DOCTYPE html>\n<html class=""no-js"">\n<head>\...",malicious,77.0,1389.0,2720.0,2032.0
9,"<!DOCTYPE html>\n<html class=""b-header--bl...",malicious,14.0,119.0,76.0,0.0


---

## <b>Q5. script 에서 src, href 속성을 가진 태그수

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- float으로 return 받기

### <span style="color:pink">[문제5] Beatuifulsoup 라이브러리를 활용 HTML code에서 \<script> 태그에서 src, href 속성을 가진 태그수를 계산하는 함수를 완성하고 결과를 확인하세요. <span>


In [201]:
soup = BeautifulSoup(row.html_code, 'html.parser')
print(str(soup.findAll("script", {'src' : True})))


AttributeError: 'Series' object has no attribute 'df'

In [231]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.
def find_src(soup):
    try:
        linkcount = len(str(soup.findAll("script", {'src' : True})))
        linkcount += len(str(soup.findAll("script", {'href' : True})))
        return linkcount # 타이틀길이 반환 함수
    except:
        return 0.0 # 타이틀이 없으면 0 반환

In [232]:
# display(df)
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다
src = []

for index, row in df.iterrows():
    soup = BeautifulSoup(row.html_code, 'html.parser')
    src.append(find_src(soup))
print(src)

[4, 424, 1054, 4, 4, 4, 600, 479, 4, 116, 164, 4, 89, 59, 4, 4, 4, 389, 236, 155, 4, 4, 485, 4, 4, 1441, 1550, 90, 333, 1650, 257, 361, 435, 230, 4, 539, 4, 4, 1106, 489]


In [233]:
# 추출한 Feature(특징) 데이터를 확인합니다.



## <b>Q6. 추가적으로 도출 가능한 Feature

- BeautifulSoup으로 html소스를 python 객체로 변환
- 함수로 구현하기
- 적절한 자료형으로 return 받기

### <span style="color:pink">[문제6] Beatuifulsoup 라이브러리를 활용 HTML code에서 추가로 만들수 있는 Feature를 찾아보고 결과를 확인하세요. <span>


In [194]:
# Feature(특징) 데이터를 추출하는 함수를 작성합니다.


In [195]:
# 데이터 프레임의 html_code 컬럼에서 Feature(특징) 데이터를 추출합니다


In [196]:
# 추출한 Feature(특징) 데이터를 확인합니다.

